In [1]:
# import libs
import pandas as pd
import pprint

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
# get titles from excels
def get_records_from_excel(file_path):
    df = pd.read_excel(file_path)
    values_set = list(df.iloc[:, 0])
    return values_set


records = get_records_from_excel("/content/drive/My Drive/project/finals/output.xlsx")
print(records)
records2 = get_records_from_excel("/content/drive/My Drive/project/finals/output2.xlsx")
print(records2)

['شارژر همراه 20000 میلی آمپر شیائومی مدل Redmi PB200LZM همراه با 2 پورت خروجی', 'شارژر همراه 10000 میلی آمپر شیائومی مدل Redmi PB100LZM همراه با 2 پورت خروجی', 'کیبورد و ماوس بی سیم شیائومی مدل WXJS01YM', 'مانیتور 19 اینچی داهوا مدل DHI-LM19-A200', 'کابل HDMI آر تی سی مدل RT17HD طول 17 متر', 'کارت حافظه microSDHC ویکو من مدل Extre600X کلاس 10 سرعت 90MBps ظرفیت 32 گیگابایت همراه با آداپتور SD', 'کارت هدیه 100.000 تومانی تیمچه طرح دوستت دارم', 'مانیتور 24 اینچی ایسوس مدل VG24VQ با کیفیت تصویر FULL HD', 'تبلت 10.1 اینچی سامسونگ مدل Galaxy TAB A 2019 LTE SM-T515 با ظرفیت 32 گیگابایت', 'تبلت 11.0 اینچی سامسونگ مدل Galaxy Tab S7 SM-T875 با ظرفیت 128 گیگابایت', 'مودم 4G/LTE قابل حمل بی سیم دی-لینک مدل DWR-910M', 'کیبورد و ماوس مخصوص بازی وریتی مدل V-KB6121GCW', 'فلش مموری سن دیسک مدل ULTRA FIT USB 3.1 CZ430 ظرفیت 64 گیگابایت با رابط USB 3.1', 'کابل HDMI مدل NV-014 طول 1.5 متر', 'شارژر همراه 10000 میلی آمپر یونیوو مدل UN10 همراه با 2 پورت خروجی', 'مانیتور 27 اینچی شیائومی مدل RMMNT27NF با کیف

In [4]:
!pip install hazm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 892.6/892.6 kB 10.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.4/4.4 MB 40.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.3/17.3 MB 44.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 53.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 238.8/238.8 kB 20.5 MB/s eta 0:00:00
  Created wheel for flashtext: filename=flashtext-2.7-py2.py3-none-any.whl size=9296 sha256=845fff3116f776d101eb21232a07a8c58e35f337d37b4991a152d0cbe2c8b32e
  Stored in directory: /root/.cache/pip/wheels/bc/be/39/c37ad168eb2ff644c9685f52554440372129450f0b8ed203dd
Successfully built flashtext
  Attempting uninstall: numpy
    Found existing installation: numpy 1.25.2
    Uninstalling numpy-1.25.2:
      Successfully uninstalled numpy-1.25.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. Thi

In [4]:
# import stopwords
import re
from nltk.corpus import stopwords
import nltk
nltk.download('stopwords')
from hazm import stopwords_list

# خواندن کلمات فارسی از فایل persion.txt
# with open('/content/drive/My Drive/project/finals/persian.txt', 'r', encoding='utf-8') as file:
#     persian_stopwords = file.read().split()

english_stopwords = set(stopwords.words('english'))


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [14]:
from hazm import word_tokenize
from hazm import Normalizer
from hazm import Lemmatizer

# Function to tokenize and lemmatize sentences
def process_sentences_to_dict(records):
    sentence_dict = {}
    normalizer = Normalizer()
    lemmatizer = Lemmatizer()

    for sentence in records:
        normalized_sentence = normalizer.normalize(sentence)
        tokens = word_tokenize(normalized_sentence)
        processed_tokens = []

        for word in tokens:
            if re.search(r'[\u0600-\u06FF]', word):  # Persian
                word = lemmatizer.lemmatize(word)
                if word not in stopwords_list():
                    processed_tokens.append(word)
            elif word.lower() not in english_stopwords:  # English
                processed_tokens.append(word)

        sentence_dict[sentence] = processed_tokens

    return sentence_dict

# Process sentences and create dictionaries
dic1 = process_sentences_to_dict(records)
dic2 = process_sentences_to_dict(records2)

In [16]:
# Calculate similarity with count similar words
equal_w = {}
l = []
df = pd.DataFrame(records2, columns=['values ex 2 / values ex 1'])

for titles in records:
    word_ex1 = [value for key, value in dic1.items() if key == titles]
    for r in records2:
        words_ex2 = [value for key, value in dic2.items() if key == r]
        i = 0
        for w1 in word_ex1[0]:
            for w2 in words_ex2[0]:
                if w1 == w2:
                    i += 1
        l.append(i)

    df[titles] = l
    l = []

df.to_excel('df2.xlsx', index=False)

In [17]:
df.head()

,values ex 2 / values ex 1,شارژر همراه 20000 میلی آمپر شیائومی مدل Redmi PB200LZM همراه با 2 پورت خروجی,شارژر همراه 10000 میلی آمپر شیائومی مدل Redmi PB100LZM همراه با 2 پورت خروجی,کیبورد و ماوس بی سیم شیائومی مدل WXJS01YM,مانیتور 19 اینچی داهوا مدل DHI-LM19-A200,کابل HDMI آر تی سی مدل RT17HD طول 17 متر,کارت حافظه microSDHC ویکو من مدل Extre600X کلاس 10 سرعت 90MBps ظرفیت 32 گیگابایت همراه با آداپتور SD,کارت هدیه 100.000 تومانی تیمچه طرح دوستت دارم,مانیتور 24 اینچی ایسوس مدل VG24VQ با کیفیت تصویر FULL HD,تبلت 10.1 اینچی سامسونگ مدل Galaxy TAB A 2019 LTE SM-T515 با ظرفیت 32 گیگابایت,...,مودم 4G/LTE قابل حمل بی سیم دی-لینک مدل DWR-910M,کیبورد و ماوس مخصوص بازی وریتی مدل V-KB6121GCW,فلش مموری سن دیسک مدل ULTRA FIT USB 3.1 CZ430 ظرفیت 64 گیگابایت با رابط USB 3.1,کابل HDMI مدل NV-014 طول 1.5 متر,شارژر همراه 10000 میلی آمپر یونیوو مدل UN10 همراه با 2 پورت خروجی,مانیتور 27 اینچی شیائومی مدل RMMNT27NF با کیفیت تصویر HD,مانیتور 24 اینچی ایسوس مدل VG248QE با کیفیت تصویر Full HD,شارژر همراه 20000 میلی آمپر یونیوو مدل UN20 همراه با 2 پورت خروجی,مانیتور 22 اینچی ال جی مدل 22MK400H-B با کیفیت تصویر Full HD,مودم روتر تی پی لینک دو آنتن ADSL2 Plus بی سیم مدل W8961N
0,کارت حافظه microSDHC ویکو من مدل Extre600X کلا...,3,3,1,1,1,19,1,1,5,...,1,1,3,1,4,1,1,3,1,1
1,کارت هدیه 50.000 تومانی تیمچه طرح روز دانشجو,0,0,0,0,0,1,7,0,1,...,0,0,2,1,0,0,0,0,0,0
2,مانیتور 27 اینچی مخصوص بازی جی پلاس مدل GGM-L2...,1,1,1,3,2,1,0,6,2,...,1,3,1,1,1,8,7,1,7,1
3,مودم روتر تی پی-لینک دو آنتن LTE بی سیم مدل Ar...,1,2,2,1,2,1,1,1,2,...,4,1,1,1,1,1,1,1,1,5
4,مودم روتر تی پی لینک N300 LTE بی سیم مدل TL-MR...,1,1,2,1,2,1,0,1,2,...,4,1,1,1,1,1,1,1,1,6


In [18]:
# Find the most similar records
most_similar_records = []

for titles in records:
    similarities = df[titles]
    max_similarity = similarities.max()
    most_similar_index = similarities.idxmax()
    most_similar_title = records2[most_similar_index]
    most_similar_records.append((titles, most_similar_title, max_similarity))

# Create a DataFrame for the results
df_results = pd.DataFrame(most_similar_records, columns=['Record from List 1', 'Most Similar Record from List 2', 'Similarity'])


In [19]:
df_results.head()

,Record from List 1,Most Similar Record from List 2,Similarity
0,شارژر همراه 20000 میلی آمپر شیائومی مدل Redmi ...,شارژر همراه 20000 میلی آمپر شیائومی مدل MI - R...,16
1,شارژر همراه 10000 میلی آمپر شیائومی مدل Redmi ...,شارژر همراه 20000 میلی آمپر شیائومی مدل MI - R...,14
2,کیبورد و ماوس بی سیم شیائومی مدل WXJS01YM,مودم روتر تی پی-لینک دو آنتن LTE بی سیم مدل Ar...,2
3,مانیتور 19 اینچی داهوا مدل DHI-LM19-A200,مانیتور 27 اینچی مخصوص بازی جی پلاس مدل GGM-L2...,3
4,کابل HDMI آر تی سی مدل RT17HD طول 17 متر,کابل HDMI آر تی سی مدل RT17HD طول 17 متر,11
